<a href="https://colab.research.google.com/github/jesminmousumy/Python-Programming/blob/main/SQL_quaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#install packages
import pandas as pd #data manipulation and analysis
import sqlite3  # small to medium data analysis

conn = sqlite3.connect(':memory:')  #establish a connection to a new SQLite database, :memory: create a temporary database
cursor = conn.cursor()   # allow us to execute sql commands

#create a 'customers' table
cursor.execute('''
CREATE TABLE customers(
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    country TEXT
)''')


#create 'orders' table
cursor.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total_amount REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)''')

#insert sample data into 'customers' table
customers_data = [
    (1, 'Mousumy','Bangladesh'),
    (2, 'Rabbani','Bangladesh'),
    (3, 'Muna', 'USA'),
    (4, 'David','Germany'),
    (5, 'Maria','Spain'),
]
cursor.executemany('INSERT INTO customers VALUES (?, ?, ?)', customers_data)


#insert sample data into 'orders' tables

orders_data = [
    (1,1,'2024-01-29',200.00),
    (2,2,'2024-05-15',250.00),
    (3,1,'2024-07-15',300.25),
    (4,3,'2024-06-15', 150.00),
    (6,5,'2024-06-05',600.00)
]
cursor.executemany('INSERT INTO orders VALUES (?, ?, ?, ?)', orders_data)

#commit the changes to the database
conn.commit()





In [14]:
#select all data from customers table
query = 'SELECT * FROM customers'
df = pd.read_sql_query(query, conn)
print(df)


   customer_id     name     country
0            1  Mousumy  Bangladesh
1            2  Rabbani  Bangladesh
2            3     Muna         USA
3            4    David     Germany
4            5    Maria       Spain


In [15]:
#select customers name who are from Bangladesh
query = 'SELECT name FROM customers WHERE country = "Bangladesh"'
df = pd.read_sql_query(query, conn)
print(df)

      name
0  Mousumy
1  Rabbani


In [19]:
#List all orders placed by Mousumy
query = '''
SELECT orders.order_id, orders.order_date, orders.total_amount
FROM orders
JOIN customers ON orders.customer_id = customers.customer_id
WHERE customers.name = 'Mousumy'
'''
df = pd.read_sql_query(query, conn)
print(df)

   order_id  order_date  total_amount
0         1  2024-01-29        200.00
1         3  2024-07-15        300.25


In [21]:
#Find the total sales made by each customers(total amount)
query = '''
SELECT customers.name, SUM(orders.total_amount) AS total_sales
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id
GROUP BY customers.name
'''
df = pd.read_sql_query(query, conn)
print(df)

      name  total_sales
0    Maria       600.00
1  Mousumy       500.25
2     Muna       150.00
3  Rabbani       250.00


In [23]:
#Get the average order value for each customer
query = '''
SELECT customers.name, AVG(orders.total_amount) AS avg_order_value
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id
GROUP BY customers.name
'''
df = pd.read_sql_query(query, conn)
print(df)

      name  avg_order_value
0    Maria          600.000
1  Mousumy          250.125
2     Muna          150.000
3  Rabbani          250.000


In [25]:
#List all customers who have placed more than one order
query = '''
SELECT customers.name, COUNT(orders.order_id) AS num_orders
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id
GROUP BY customers.name
HAVING COUNT(orders.order_id) > 1
'''
df = pd.read_sql_query(query, conn)
print(df)


      name  num_orders
0  Mousumy           2


In [26]:
#Retrieve the most recent order for each customer
query = '''
SELECT customers.name, MAX(orders.order_date) AS most_recent_order_date
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id
GROUP BY customers.name
'''
df = pd.read_sql_query(query, conn)
print(df)

      name most_recent_order_date
0    Maria             2024-06-05
1  Mousumy             2024-07-15
2     Muna             2024-06-15
3  Rabbani             2024-05-15
